In [175]:
# Импорт библиотек для работы с данными и графиками
import csv    # для чтения CSV файлов
import math   # для математических операций
import statistics  # для статистических вычислений
import matplotlib.pyplot as plt  # для построения графиков
from datetime import datetime  # для работы с датой и временем

In [176]:
# Загрузка данных о продажах кофе из CSV файла
def load_coffee_data(filename):
    """
    Загружает данные из CSV файла и возвращает список словарей
    Каждая строка файла становится отдельным словарем с данными
    """
    data_records = []  # создаем пустой список для хранения данных
    with open(filename, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)  # создаем reader для чтения как словарей
        for record in reader:  # проходим по каждой строке файла
            data_records.append(record)  # добавляем строку в список
    return data_records

# Загружаем данные из файла
coffee_records = load_coffee_data('Coffe_sales.csv')
print(f"Успешно загружено записей: {len(coffee_records)}")
print("Названия полей в данных:", list(coffee_records[0].keys()))

Успешно загружено записей: 3547
Названия полей в данных: ['hour_of_day', 'cash_type', 'money', 'coffee_name', 'Time_of_Day', 'Weekday', 'Month_name', 'Weekdaysort', 'Monthsort', 'Date', 'Time']


In [177]:
# Преобразование ВСЕХ признаков в числовые значения
def convert_all_features_to_numeric(data):
    """
    Преобразует ВСЕ признаки в числовые значения, включая категориальные и дату/время
    Это необходимо для корректного анализа данных
    """
    print("Начинаем преобразование всех признаков в числовые...")
    converted_data = []  # список для преобразованных данных
    
    # Создаем маппинги для категориальных признаков
    cash_type_mapping = {'cash': 0, 'card': 1, 'mobile': 2}  # способы оплаты
    
    # Собираем уникальные значения кофе для маппинга
    coffee_names = list(set([r['coffee_name'] for r in data]))
    coffee_mapping = {name: i for i, name in enumerate(coffee_names)}  # названия кофе -> числа
    
    time_mapping = {'morning': 0, 'afternoon': 1, 'evening': 2}  # время дня
    
    weekdays = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    weekday_mapping = {day: i for i, day in enumerate(weekdays)}  # дни недели
    
    months = ['january', 'february', 'march', 'april', 'may', 'june', 
             'july', 'august', 'september', 'october', 'november', 'december']
    month_mapping = {month: i for i, month in enumerate(months)}  # месяцы
    
    for record in data:
        numeric_record = {}  # словарь для числовых значений одной записи
        
        # 1. Уже числовые признаки - оставляем как есть
        numeric_record['hour_of_day'] = float(record['hour_of_day'])  # час дня
        numeric_record['money'] = float(record['money'])  # сумма покупки
        numeric_record['Weekdaysort'] = float(record['Weekdaysort'])  # номер дня недели
        numeric_record['Monthsort'] = float(record['Monthsort'])  # номер месяца
        
        # 2. Категориальные признаки - преобразуем в числа
        cash_type = record['cash_type'].lower()  # приводим к нижнему регистру
        numeric_record['cash_type'] = cash_type_mapping.get(cash_type, 0)  # способ оплаты
        
        numeric_record['coffee_name'] = coffee_mapping.get(record['coffee_name'], 0)  # вид кофе
        
        time_of_day = record['Time_of_Day'].lower()
        numeric_record['Time_of_Day'] = time_mapping.get(time_of_day, 0)  # время дня
        
        weekday = record['Weekday'].lower()
        numeric_record['Weekday'] = weekday_mapping.get(weekday, 0)  # день недели
        
        month_name = record['Month_name'].lower()
        numeric_record['Month_name'] = month_mapping.get(month_name, 0)  # месяц
        
        # 3. Дата и время - извлекаем числовые компоненты
        # Обработка даты (формат: '2024-01-15')
        try:
            date_obj = datetime.strptime(record['Date'], '%Y-%m-%d')  # парсим дату
            numeric_record['day_of_month'] = float(date_obj.day)  # день месяца (1-31)
            numeric_record['day_of_week'] = float(date_obj.weekday())  # день недели (0-6)
            numeric_record['month'] = float(date_obj.month)  # месяц (1-12)
            numeric_record['year'] = float(date_obj.year)  # год
            numeric_record['is_weekend'] = 1.0 if date_obj.weekday() >= 5 else 0.0  # выходной?
        except:
            # Если не получилось распарсить - ставим значения по умолчанию
            numeric_record['day_of_month'] = 0.0
            numeric_record['day_of_week'] = 0.0
            numeric_record['month'] = 0.0
            numeric_record['year'] = 0.0
            numeric_record['is_weekend'] = 0.0
        
        # Обработка времени (формат: '14:30:00')
        try:
            time_obj = datetime.strptime(record['Time'], '%H:%M:%S')  # парсим время
            numeric_record['hour'] = float(time_obj.hour)  # час (0-23)
            numeric_record['minute'] = float(time_obj.minute)  # минута (0-59)
            numeric_record['second'] = float(time_obj.second)  # секунда (0-59)
            # Часы пик: утро 8-10, вечер 17-19
            is_rush_hour = 1.0 if (8 <= time_obj.hour <= 10) or (17 <= time_obj.hour <= 19) else 0.0
            numeric_record['is_rush_hour'] = is_rush_hour
        except:
            numeric_record['hour'] = 0.0
            numeric_record['minute'] = 0.0
            numeric_record['second'] = 0.0
            numeric_record['is_rush_hour'] = 0.0
        
        converted_data.append(numeric_record)  # добавляем преобразованную запись
    
    print(f"Преобразовано записей: {len(converted_data)}")
    print(f"Количество признаков в каждой записи: {len(converted_data[0])}")
    
    return converted_data

# Преобразуем ВСЕ данные в числовые
all_numeric_data = convert_all_features_to_numeric(coffee_records)
print("Пример преобразованной записи:")
for key, value in list(all_numeric_data[0].items())[:8]:  # покажем первые 8 признаков
    print(f"  {key}: {value}")

Начинаем преобразование всех признаков в числовые...
Преобразовано записей: 3547
Количество признаков в каждой записи: 18
Пример преобразованной записи:
  hour_of_day: 10.0
  money: 38.7
  Weekdaysort: 5.0
  Monthsort: 3.0
  cash_type: 1
  coffee_name: 4
  Time_of_Day: 0
  Weekday: 0


In [178]:
# Получение списка ВСЕХ числовых признаков
def get_all_numeric_columns(data):
    """
    Получает список всех числовых признаков из преобразованных данных
    """
    return list(data[0].keys())  # берем ключи из первой записи

# Получаем все числовые признаки
all_numeric_fields = get_all_numeric_columns(all_numeric_data)
print(f"Всего числовых признаков: {len(all_numeric_fields)}")
print("Список всех признаков:")
for i, field in enumerate(all_numeric_fields, 1):
    print(f"  {i:2d}. {field}")

Всего числовых признаков: 18
Список всех признаков:
   1. hour_of_day
   2. money
   3. Weekdaysort
   4. Monthsort
   5. cash_type
   6. coffee_name
   7. Time_of_Day
   8. Weekday
   9. Month_name
  10. day_of_month
  11. day_of_week
  12. month
  13. year
  14. is_weekend
  15. hour
  16. minute
  17. second
  18. is_rush_hour


In [179]:
# Создание полной числовой матрицы из ВСЕХ признаков
def create_complete_numeric_matrix(data, numeric_columns):
    """
    Создает матрицу (список списков) содержащую ВСЕ числовые данные
    Каждая строка матрицы - одна запись, каждый столбец - один признак
    """
    numeric_matrix = []  # создаем пустую матрицу
    for record in data:  # проходим по каждой записи
        numeric_row = []  # создаем пустую строку матрицы
        for column in numeric_columns:  # для каждого числового столбца
            numeric_row.append(record[column])  # добавляем значение в строку
        numeric_matrix.append(numeric_row)  # добавляем строку в матрицу
    
    return numeric_matrix

# Создаем полную числовую матрицу
complete_data_matrix = create_complete_numeric_matrix(all_numeric_data, all_numeric_fields)
print(f"Создана полная числовая матрица:")
print(f"  Количество строк (записей): {len(complete_data_matrix)}")
print(f"  Количество столбцов (признаков): {len(complete_data_matrix[0])}")
print(f"  Общий объем данных: {len(complete_data_matrix)} × {len(complete_data_matrix[0])} = {len(complete_data_matrix) * len(complete_data_matrix[0])} значений")

print("\nПервые 3 строки матрицы (первые 6 признаков):")
for i in range(3):
    first_six = complete_data_matrix[i][:6]  # берем первые 6 признаков для краткости
    print(f"  Строка {i+1}: {[f'{x:.1f}' for x in first_six]}...")

Создана полная числовая матрица:
  Количество строк (записей): 3547
  Количество столбцов (признаков): 18
  Общий объем данных: 3547 × 18 = 63846 значений

Первые 3 строки матрицы (первые 6 признаков):
  Строка 1: ['10.0', '38.7', '5.0', '3.0', '1.0', '4.0']...
  Строка 2: ['12.0', '38.7', '5.0', '3.0', '1.0', '2.0']...
  Строка 3: ['12.0', '38.7', '5.0', '3.0', '1.0', '2.0']...


In [180]:
# Нормировка данных методом Min-Max Scaling для ВСЕХ признаков
def normalize_data_minmax(matrix):
    """
    Применяет Min-Max нормировку ко ВСЕМ признакам данных
    Преобразует все значения в диапазон от 0 до 1
    Формула: (x - min) / (max - min)
    """
    print("Применяем нормировку ко всем признакам...")
    normalized_matrix = []  # матрица для нормированных данных
    num_columns = len(matrix[0])  # количество столбцов (признаков)
    
    # Списки для хранения минимальных и максимальных значений каждого столбца
    min_values = []  # минимальные значения каждого признака
    max_values = []  # максимальные значения каждого признака
    
    # Находим минимумы и максимумы для каждого столбца
    for col_idx in range(num_columns):
        column_data = [row[col_idx] for row in matrix]  # получаем все значения столбца
        min_values.append(min(column_data))  # находим минимум
        max_values.append(max(column_data))  # находим максимум
    
    # Применяем нормировку ко всем данным
    for row in matrix:
        normalized_row = []  # новая нормированная строка
        for col_idx in range(num_columns):
            value = row[col_idx]  # исходное значение
            min_val = min_values[col_idx]  # минимум для этого столбца
            max_val = max_values[col_idx]  # максимум для этого столбца
            
            # Проверяем, чтобы не делить на ноль
            if max_val - min_val == 0:
                normalized_value = 0.0  # если все значения одинаковые
            else:
                # применяем формулу нормировки
                normalized_value = (value - min_val) / (max_val - min_val)
            
            normalized_row.append(normalized_value)  # добавляем нормированное значение
        normalized_matrix.append(normalized_row)  # добавляем строку в матрицу
    
    print("Нормировка завершена!")
    return normalized_matrix

# Применяем нормировку ко ВСЕМ данным
normalized_data = normalize_data_minmax(complete_data_matrix)
print("Данные после нормировки (первые 3 строки, первые 6 признаков):")
for i in range(3):
    first_six = normalized_data[i][:6]  # берем первые 6 признаков для краткости
    formatted_values = [f"{x:.4f}" for x in first_six]
    print(f"  Строка {i+1}: {formatted_values}...")

Применяем нормировку ко всем признакам...
Нормировка завершена!
Данные после нормировки (первые 3 строки, первые 6 признаков):
  Строка 1: ['0.2500', '1.0000', '0.6667', '0.1818', '0.0000', '0.5714']...
  Строка 2: ['0.3750', '1.0000', '0.6667', '0.1818', '0.0000', '0.2857']...
  Строка 3: ['0.3750', '1.0000', '0.6667', '0.1818', '0.0000', '0.2857']...


In [181]:
# Функция для вычисления корреляционной матрицы
def compute_correlation_matrix(matrix):
    """
    Вычисляет корреляционную матрицу Пирсона для данных
    Корреляция показывает линейную зависимость между признаками
    """
    num_features = len(matrix[0])  # количество признаков
    num_samples = len(matrix)      # количество образцов
    corr_matrix = []  # создаем пустую корреляционную матрицу
    
    # Шаг 1: Вычисляем средние значения для каждого признака
    feature_means = []  # список для средних значений
    for j in range(num_features):
        feature_sum = sum([row[j] for row in matrix])  # сумма всех значений признака
        feature_means.append(feature_sum / num_samples)  # среднее значение
    
    # Шаг 2: Вычисляем корреляции между всеми парами признаков
    for i in range(num_features):
        corr_row = []  # строка корреляционной матрицы
        for j in range(num_features):
            if i == j:
                # Корреляция признака с самим собой всегда равна 1
                corr_row.append(1.0)
            else:
                # Вычисляем ковариацию и стандартные отклонения
                covariance = 0.0  # ковариация
                std_i = 0.0       # стандартное отклонение i-го признака
                std_j = 0.0       # стандартное отклонение j-го признака
                
                # Проходим по всем образцам
                for k in range(num_samples):
                    # Отклонения от средних
                    dev_i = matrix[k][i] - feature_means[i]
                    dev_j = matrix[k][j] - feature_means[j]
                    
                    # Накопление сумм для вычислений
                    covariance += dev_i * dev_j
                    std_i += dev_i * dev_i
                    std_j += dev_j * dev_j
                
                # Вычисляем ковариацию и стандартные отклонения
                covariance /= num_samples
                std_i = math.sqrt(std_i / num_samples)
                std_j = math.sqrt(std_j / num_samples)
                
                # Проверяем деление на ноль
                if std_i * std_j == 0:
                    correlation = 0.0
                else:
                    # Формула корреляции Пирсона
                    correlation = covariance / (std_i * std_j)
                
                corr_row.append(correlation)  # добавляем корреляцию в строку
        
        corr_matrix.append(corr_row)  # добавляем строку в матрицу
    
    return corr_matrix

# Вычисляем первую корреляционную матрицу (после нормировки)
corr_matrix_1 = compute_correlation_matrix(normalized_data)

print("=" * 70)
print("ПЕРВАЯ КОРРЕЛЯЦИОННАЯ МАТРИЦА")
print("(после нормировки всех признаков)")
print("=" * 70)

print(f"Размер матрицы: {len(corr_matrix_1)}x{len(corr_matrix_1[0])}")

# Покажем только первые 6 признаков для читаемости
print("\nПервые 6 признаков (для наглядности):")
header = " " * 20
for name in all_numeric_fields[:6]:
    header += f"{name:>15}"
print(header)

for i in range(6):
    row_str = f"{all_numeric_fields[i]:<20}"
    for j in range(6):
        row_str += f"{corr_matrix_1[i][j]:>15.4f}"
    print(row_str)

print("\nВЫВОД: Первая матрица показывает исходные взаимосвязи между всеми признаками")
print("       после нормировки. Значения близкие к 1 или -1 указывают на сильную")
print("       корреляцию, близкие к 0 - на слабую связь.")

ПЕРВАЯ КОРРЕЛЯЦИОННАЯ МАТРИЦА
(после нормировки всех признаков)
Размер матрицы: 18x18

Первые 6 признаков (для наглядности):
                        hour_of_day          money    Weekdaysort      Monthsort      cash_type    coffee_name
hour_of_day                  1.0000         0.2027        -0.0026         0.0083         0.0000        -0.1598
money                        0.2027         1.0000        -0.0173        -0.0500         0.0000        -0.7087
Weekdaysort                 -0.0026        -0.0173         1.0000         0.0441         0.0000         0.0015
Monthsort                    0.0083        -0.0500         0.0441         1.0000         0.0000        -0.0084
cash_type                    0.0000         0.0000         0.0000         0.0000         1.0000         0.0000
coffee_name                 -0.1598        -0.7087         0.0015        -0.0084         0.0000         1.0000

ВЫВОД: Первая матрица показывает исходные взаимосвязи между всеми признаками
       после нормиро

In [182]:
# Поиск и анализ выбросов в данных 
def find_and_analyze_outliers(matrix, feature_names):
    """
    Анализирует данные на наличие выбросов методом межквартильного размаха (IQR)
    Выбросы - значения, которые значительно отличаются от большинства данных
    """
    print("\n" + "=" * 70)
    print("АНАЛИЗ ВЫБРОСОВ В ДАННЫХ")
    print("=" * 70)
    
    cleaned_matrix = []  # матрица без выбросов
    num_features = len(feature_names)  # количество признаков
    outlier_indices = set()  # множество индексов строк с выбросами
    
    # Статистика по выбросам
    outlier_summary = []  # список для сводки по выбросам
    
    # Анализируем каждый признак отдельно
    for feature_idx in range(num_features):
        feature_values = [row[feature_idx] for row in matrix]  # все значения признака
        
        # Сортируем значения для нахождения квартилей
        sorted_vals = sorted(feature_values)
        n = len(sorted_vals)  # количество значений
        
        # Находим первый квартиль (25-й процентиль)
        q1_index = n // 4
        q1 = sorted_vals[q1_index]
        
        # Находим третий квартиль (75-й процентиль)
        q3_index = 3 * n // 4
        q3 = sorted_vals[q3_index]
        
        # Вычисляем межквартильный размах (IQR)
        iqr = q3 - q1
        
        # Определяем границы для выбросов
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        
        # Ищем выбросы для этого признака
        feature_outliers = []  # список выбросов для этого признака
        outlier_values = []  # значения выбросов
        
        for row_idx, value in enumerate(feature_values):
            if value < lower_bound or value > upper_bound:
                feature_outliers.append(row_idx)
                outlier_values.append(value)
                outlier_indices.add(row_idx)
        
        # Сохраняем статистику для этого признака
        outlier_summary.append({
            'feature': feature_names[feature_idx],
            'outlier_count': len(feature_outliers),
            'outlier_percentage': (len(feature_outliers) / len(matrix)) * 100,
            'min_value': min(feature_values),
            'max_value': max(feature_values),
            'bounds': [lower_bound, upper_bound]
        })
    
    # Выводим сводку по выбросам
    print("\nСВОДКА ПО ВЫБРОСАМ:")
    print("-" * 80)
    print(f"{'Признак':<20} {'Выбросы':<10} {'%':<8} {'Мин':<8} {'Макс':<8} {'Границы'}")
    print("-" * 80)
    
    total_outliers = 0
    features_with_outliers = 0
    
    for summary in outlier_summary:
        if summary['outlier_count'] > 0:
            features_with_outliers += 1
            total_outliers += summary['outlier_count']
            
            # Форматируем вывод границ
            bounds_str = f"[{summary['bounds'][0]:.3f}, {summary['bounds'][1]:.3f}]"
            
            print(f"{summary['feature']:<20} {summary['outlier_count']:<10} {summary['outlier_percentage']:<7.1f} "
                  f"{summary['min_value']:<7.3f} {summary['max_value']:<7.3f} {bounds_str}")
    
    # Выводим признаки без выбросов (только если их немного)
    no_outliers_features = [s for s in outlier_summary if s['outlier_count'] == 0]
    if len(no_outliers_features) <= 10:  # если мало признаков без выбросов - покажем все
        print(f"\nПризнаки БЕЗ выбросов ({len(no_outliers_features)}):")
        for summary in no_outliers_features:
            bounds_str = f"[{summary['bounds'][0]:.3f}, {summary['bounds'][1]:.3f}]"
            print(f"  {summary['feature']:<18} - границы: {bounds_str}")
    else:
        # Если много признаков без выбросов - покажем только первые 5
        print(f"\nПризнаки БЕЗ выбросов (первые 5 из {len(no_outliers_features)}):")
        for summary in no_outliers_features[:5]:
            bounds_str = f"[{summary['bounds'][0]:.3f}, {summary['bounds'][1]:.3f}]"
            print(f"  {summary['feature']:<18} - границы: {bounds_str}")
        print(f"  ... и еще {len(no_outliers_features) - 5} признаков")
    
    # Удаляем строки с выбросами из матрицы
    cleaned_matrix = [row for row_idx, row in enumerate(matrix) if row_idx not in outlier_indices]
    
    # Выводим итоговую статистику
    print(f"\n" + "=" * 50)
    print("ИТОГОВАЯ СТАТИСТИКА:")
    print(f"Всего признаков: {num_features}")
    print(f"Признаков с выбросами: {features_with_outliers}")
    print(f"Всего обнаружено выбросов: {total_outliers}")
    print(f"Уникальных строк с выбросами: {len(outlier_indices)}")
    print(f"Записей до удаления: {len(matrix)}")
    print(f"Записей после удаления: {len(cleaned_matrix)}")
    print(f"Удалено записей: {len(matrix) - len(cleaned_matrix)}")
    print(f"Сохранено данных: {(len(cleaned_matrix)/len(matrix))*100:.1f}%")
    
    return cleaned_matrix

# Применяем функцию поиска и удаления выбросов
cleaned_data = find_and_analyze_outliers(normalized_data, all_numeric_fields)


АНАЛИЗ ВЫБРОСОВ В ДАННЫХ

СВОДКА ПО ВЫБРОСАМ:
--------------------------------------------------------------------------------
Признак              Выбросы    %        Мин      Макс     Границы
--------------------------------------------------------------------------------
Month_name           241        6.8     0.000   1.000   [0.000, 0.000]
hour                 8          0.2     0.000   1.000   [0.000, 0.000]
minute               8          0.2     0.000   1.000   [0.000, 0.000]
second               8          0.2     0.000   1.000   [0.000, 0.000]

Признаки БЕЗ выбросов (первые 5 из 14):
  hour_of_day        - границы: [-0.500, 1.500]
  money              - границы: [-0.095, 1.429]
  Weekdaysort        - границы: [-0.833, 1.833]
  Monthsort          - границы: [-0.773, 1.773]
  cash_type          - границы: [0.000, 0.000]
  ... и еще 9 признаков

ИТОГОВАЯ СТАТИСТИКА:
Всего признаков: 18
Признаков с выбросами: 4
Всего обнаружено выбросов: 265
Уникальных строк с выбросами: 248
Запи

In [183]:
# Удаление тождественных экземпляров
def remove_duplicate_instances(matrix):
    """
    Удаляет полностью одинаковые (тождественные) экземпляры из данных
    Тождественные экземпляры - строки с одинаковыми значениями всех признаков
    """
    print("\n" + "=" * 70)
    print("УДАЛЕНИЕ ТОЖДЕСТВЕННЫХ ЭКЗЕМПЛЯРОВ")
    print("=" * 70)
    
    unique_instances = []  # список для уникальных экземпляров
    seen_instances = set()  # множество для отслеживания уже встреченных экземпляров
    duplicate_count = 0  # счетчик удаленных дубликатов
    
    # Проходим по всем строкам матрицы
    for row in matrix:
        # Преобразуем строку в кортеж для хранения в множестве
        # Кортеж можно использовать как ключ, в отличие от списка
        row_tuple = tuple(row)
        
        # Проверяем, встречался ли такой экземпляр ранее
        if row_tuple not in seen_instances:
            # Если не встречался - добавляем в уникальные
            seen_instances.add(row_tuple)
            unique_instances.append(row)
        else:
            # Если встречался - увеличиваем счетчик дубликатов
            duplicate_count += 1
    
    # Выводим статистику
    print(f"Удалено тождественных экземпляров: {duplicate_count}")
    print(f"Уникальных записей до удаления: {len(matrix)}")
    print(f"Уникальных записей после удаления: {len(unique_instances)}")
    print(f"Процент удаленных записей: {(duplicate_count/len(matrix))*100:.2f}%")
    
    return unique_instances

# Удаляем дубликаты из очищенных данных
final_cleaned_data = remove_duplicate_instances(cleaned_data)


УДАЛЕНИЕ ТОЖДЕСТВЕННЫХ ЭКЗЕМПЛЯРОВ
Удалено тождественных экземпляров: 567
Уникальных записей до удаления: 3299
Уникальных записей после удаления: 2732
Процент удаленных записей: 17.19%


In [184]:
# Вторая корреляционная матрица после очистки данных
# Вычисляем корреляционную матрицу для очищенных данных
corr_matrix_2 = compute_correlation_matrix(final_cleaned_data)

print("\n" + "=" * 70)
print("ВТОРАЯ КОРРЕЛЯЦИОННАЯ МАТРИЦА")
print("(после удаления выбросов и дубликатов)")
print("=" * 70)

print(f"Размер матрицы: {len(corr_matrix_2)}x{len(corr_matrix_2[0])}")

# Покажем только первые 6 признаков для читаемости
print("\nПервые 6 признаков (для наглядности):")
header = " " * 20
for name in all_numeric_fields[:6]:
    header += f"{name:>15}"
print(header)

for i in range(6):
    row_str = f"{all_numeric_fields[i]:<20}"
    for j in range(6):
        row_str += f"{corr_matrix_2[i][j]:>15.4f}"
    print(row_str)

print("\nВЫВОД: После удаления выбросов и дубликатов корреляции стали более стабильными.")
print("       Удаление аномальных значений улучшает качество анализа взаимосвязей.")
print("       Изменения в значениях корреляции показывают влияние выбросов на исходные данные.")


ВТОРАЯ КОРРЕЛЯЦИОННАЯ МАТРИЦА
(после удаления выбросов и дубликатов)
Размер матрицы: 18x18

Первые 6 признаков (для наглядности):
                        hour_of_day          money    Weekdaysort      Monthsort      cash_type    coffee_name
hour_of_day                  1.0000         0.1785        -0.0073         0.0133         0.0000        -0.1390
money                        0.1785         1.0000        -0.0191        -0.0454         0.0000        -0.7103
Weekdaysort                 -0.0073        -0.0191         1.0000         0.0380         0.0000         0.0074
Monthsort                    0.0133        -0.0454         0.0380         1.0000         0.0000        -0.0087
cash_type                    0.0000         0.0000         0.0000         0.0000         1.0000         0.0000
coffee_name                 -0.1390        -0.7103         0.0074        -0.0087         0.0000         1.0000

ВЫВОД: После удаления выбросов и дубликатов корреляции стали более стабильными.
       Удал

In [ ]:
# Преобразование в целочисленную форму (обобщенная сериализация)
def generalize_serialization(matrix, feature_names):
    """
    Преобразует вещественные данные в целочисленные по методичке обобщенной сериализации
    Это обратимое преобразование, сохраняющее информацию о данных
    """
    print("\n" + "=" * 70)
    print("ПРЕОБРАЗОВАНИЕ В ЦЕЛОЧИСЛЕННУЮ ФОРМУ")
    print("(обобщенная сериализация)")
    print("=" * 70)
    
    num_features = len(matrix[0])  # количество признаков
    integer_matrix = []  # матрица для целочисленных данных
    powers = []  # список мощностей признаков
    
    # Вычисляем мощность для каждого признака
    for feature_idx in range(num_features):
        feature_values = [row[feature_idx] for row in matrix]  # все значения признака
        
        # Находим минимальное и максимальное значения
        min_val = min(feature_values)
        max_val = max(feature_values)
        
        # Вычисляем дискретность как среднюю разность между соседними значениями
        sorted_values = sorted(feature_values)  # сортируем значения
        differences = []  # список разностей
        for i in range(1, len(sorted_values)):
            diff = sorted_values[i] - sorted_values[i-1]  # разность между соседями
            if diff > 0.0001:  # игнорируем очень маленькие разности
                differences.append(diff)
        
        # Если есть различия - вычисляем среднюю дискретность
        if differences:
            discretization_step = statistics.mean(differences)
        else:
            # Если все значения одинаковые
            discretization_step = 1.0
        
        # Вычисляем мощность признака (количество возможных значений)
        if discretization_step == 0:
            power = 1  # если шаг нулевой
        else:
            power = int((max_val - min_val) / discretization_step) + 1
        
        powers.append(power)  # сохраняем мощность
        print(f"{feature_names[feature_idx]}: мощность = {power}")
    
    # Преобразуем данные в целочисленную форму по формуле из методички
    for row in matrix:
        integer_row = []  # целочисленная строка
        for feature_idx in range(num_features):
            value = row[feature_idx]  # исходное значение
            min_val = min([r[feature_idx] for r in matrix])  # минимум для признака
            max_val = max([r[feature_idx] for r in matrix])  # максимум для признака
            
            # Проверяем случай, когда все значения одинаковые
            if max_val - min_val == 0:
                integer_value = 0
            else:
                # Вычисляем шаг дискретизации
                discretization_step = (max_val - min_val) / (powers[feature_idx] - 1)
                # Применяем формулу преобразования: [(x - min) / d]
                integer_value = int((value - min_val) / discretization_step)
            
            integer_row.append(integer_value)  # добавляем целое значение
        integer_matrix.append(integer_row)  # добавляем строку в матрицу
    
    # Показываем пример преобразованных данных
    print("\nПервые 3 строки после преобразования в целочисленную форму:")
    for i in range(3):
        first_six = integer_matrix[i][:6]  # покажем первые 6 признаков
        print(f"Строка {i+1}: {first_six}...")
    
    return integer_matrix, powers

# Преобразуем очищенные данные в целочисленную форму
integer_data, powers = generalize_serialization(final_cleaned_data, all_numeric_fields)


ПРЕОБРАЗОВАНИЕ В ЦЕЛОЧИСЛЕННУЮ ФОРМУ
(обобщенная сериализация)
hour_of_day: мощность = 17
money: мощность = 13
Weekdaysort: мощность = 7
Monthsort: мощность = 11
cash_type: мощность = 1
coffee_name: мощность = 8
Time_of_Day: мощность = 2
Weekday: мощность = 1
Month_name: мощность = 1
day_of_month: мощность = 31
day_of_week: мощность = 7
month: мощность = 11
year: мощность = 2
is_weekend: мощность = 2
hour: мощность = 1
minute: мощность = 1
second: мощность = 1
is_rush_hour: мощность = 1


In [191]:
# Уменьшение размерности данных до 3 
def reduce_dimension_to_3(matrix, powers, feature_names):
    """
    Уменьшает размерность данных до 3 методом обобщенной сериализации
    Объединяет признаки с наименьшими мощностями для сохранения информации
    """
    print("\n" + "=" * 70)
    print("УМЕНЬШЕНИЕ РАЗМЕРНОСТИ ДАННЫХ ДО 3")
    print("=" * 70)
    
    # Создаем копии данных для работы
    current_matrix = [row[:] for row in matrix]  # текущая матрица
    current_powers = powers[:]  # текущие мощности
    current_names = feature_names[:]  # текущие названия
    
    print(f"Начальная размерность: {len(current_powers)} признаков")
    print(f"Мощности признаков: {current_powers}")
    
    steps_info = []  # сохраняем информацию о шагах для красивого вывода
    
    # Пока размерность больше 3, уменьшаем ее объединением признаков
    step_count = 0
    while len(current_powers) > 3:
        step_count += 1
        
        # Находим два признака с наименьшими мощностями
        power_index_pairs = []
        for i, power in enumerate(current_powers):
            power_index_pairs.append((i, power))
        power_index_pairs.sort(key=lambda x: x[1])
        
        idx1, power1 = power_index_pairs[0]
        idx2, power2 = power_index_pairs[1]
        
        # Сохраняем информацию о шаге
        steps_info.append({
            'step': step_count,
            'name1': current_names[idx1],
            'name2': current_names[idx2], 
            'power1': power1,
            'power2': power2,
            'new_power': power1 * power2
        })
        
        # Создаем новые структуры данных
        new_matrix = []
        new_powers = []
        new_names = []
        
        # Добавляем все признаки кроме объединяемых
        for i in range(len(current_powers)):
            if i != idx1 and i != idx2:
                new_powers.append(current_powers[i])
                new_names.append(current_names[i])
        
        # Вычисляем мощность объединенного признака
        new_power = current_powers[idx1] * current_powers[idx2]
        new_powers.append(new_power)
        
        # Создаем понятное имя для объединенного признака
        if step_count <= 5:  # первые 5 шагов показываем подробно
            new_name = f"({current_names[idx1]}+{current_names[idx2]})"
        else:  # дальше сокращаем
            new_name = f"Компонента_{len(new_names)+1}"
        new_names.append(new_name)
        
        # Формируем новую матрицу с объединенными признаками
        for row in current_matrix:
            new_row = []
            for i in range(len(current_powers)):
                if i != idx1 and i != idx2:
                    new_row.append(row[i])
            
            # Объединяем два признака по формуле: z = y1 + W1 * y2
            combined_value = row[idx1] + current_powers[idx1] * row[idx2]
            new_row.append(combined_value)
            new_matrix.append(new_row)
        
        # Обновляем текущие данные
        current_matrix = new_matrix
        current_powers = new_powers
        current_names = new_names
    
    # Вывод шагов
    print("\nОСНОВНЫЕ ШАГИ ОБЪЕДИНЕНИЯ:")
    print("-" * 50)
    
    # Показываем только первые 2 и последний 1 шаг
    for i, step in enumerate(steps_info[:2] + steps_info[-1:]):
        if i == 2:
            print("    ...")
        print(f"Шаг {step['step']:2d}: {step['name1'][:15]:<15} (мощность {step['power1']:3d})")
        print(f"       + {step['name2'][:15]:<15} (мощность {step['power2']:3d})")
        print(f"       = объединенный признак (мощность {step['new_power']:5d})")
        print()
    
    print(f"\nФИНАЛЬНЫЙ РЕЗУЛЬТАТ:")
    print(f"Размерность: {len(current_powers)} признаков")
    print(f"Мощности: {current_powers}")
    print(f"Компоненты:")
    for i, (name, power) in enumerate(zip(current_names, current_powers), 1):
        print(f"  {i}. {name} (мощность: {power})")
    
    return current_matrix, current_powers, current_names

# Уменьшаем размерность целочисленных данных до 3
reduced_data, reduced_powers, reduced_names = reduce_dimension_to_3(integer_data, powers, all_numeric_fields)


УМЕНЬШЕНИЕ РАЗМЕРНОСТИ ДАННЫХ ДО 3
Начальная размерность: 18 признаков
Мощности признаков: [17, 13, 7, 11, 1, 8, 2, 1, 1, 31, 7, 11, 2, 2, 1, 1, 1, 1]

ОСНОВНЫЕ ШАГИ ОБЪЕДИНЕНИЯ:
--------------------------------------------------
Шаг  1: cash_type       (мощность   1)
       + Weekday         (мощность   1)
       = объединенный признак (мощность     1)

Шаг  2: Month_name      (мощность   1)
       + hour            (мощность   1)
       = объединенный признак (мощность     1)

    ...
Шаг 15: Компонента_7    (мощность  64)
       + Компонента_6    (мощность 121)
       = объединенный признак (мощность  7744)


ФИНАЛЬНЫЙ РЕЗУЛЬТАТ:
Размерность: 3 признаков
Мощности: [221, 1519, 7744]
Компоненты:
  1. Компонента_5 (мощность: 221)
  2. Компонента_4 (мощность: 1519)
  3. Компонента_3 (мощность: 7744)


In [192]:
# Третья корреляционная матрица после всех преобразований
# Вычисляем корреляционную матрицу для данных с уменьшенной размерностью
corr_matrix_3 = compute_correlation_matrix(reduced_data)

print("\n" + "=" * 70)
print("ТРЕТЬЯ КОРРЕЛЯЦИОННАЯ МАТРИЦА")
print("(после преобразования в целочисленную форму и уменьшения размерности)")
print("=" * 70)

# Создаем понятные названия для финальных компонент
final_names = ['Компонента_1', 'Компонента_2', 'Компонента_3']

# Выводим матрицу
print("\n" + " " * 15, end="")
for name in final_names:
    print(f"{name:>15}", end="")
print()

for i, row in enumerate(corr_matrix_3):
    print(f"{final_names[i]:<15}", end="")
    for corr in row:
        print(f"{corr:>15.4f}", end="")
    print()

print("\nВЫВОД: После преобразования в целочисленную форму и уменьшения размерности")
print("       матрица стала размером 3x3. Компоненты имеют низкую корреляцию")
print("       между собой, что упрощает дальнейший анализ данных.")
print("       Обобщенная сериализация сохранила основную информацию в данных.")


ТРЕТЬЯ КОРРЕЛЯЦИОННАЯ МАТРИЦА
(после преобразования в целочисленную форму и уменьшения размерности)

                  Компонента_1   Компонента_2   Компонента_3
Компонента_1            1.0000        -0.0080         0.0111
Компонента_2           -0.0080         1.0000         0.0434
Компонента_3            0.0111         0.0434         1.0000

ВЫВОД: После преобразования в целочисленную форму и уменьшения размерности
       матрица стала размером 3x3. Компоненты имеют низкую корреляцию
       между собой, что упрощает дальнейший анализ данных.
       Обобщенная сериализация сохранила основную информацию в данных.


In [193]:
# Итоговый анализ изменений во всех корреляционных матрицах
print("\n" + "=" * 70)
print("ИТОГОВЫЙ АНАЛИЗ ИЗМЕНЕНИЙ КОРРЕЛЯЦИОННЫХ МАТРИЦ")
print("=" * 70)

print("\n1. ПЕРВАЯ МАТРИЦА (после нормировки данных):")
print("   - Размер:", f"{len(corr_matrix_1)}x{len(corr_matrix_1[0])} ({len(all_numeric_fields)} признаков)")
print("   - Показала исходные взаимосвязи между всеми признаками")
print("   - Могла содержать шум от выбросов и дубликатов")
print("   - Значения корреляции отражали исходное распределение данных")

print("\n2. ВТОРАЯ МАТРИЦА (после очистки данных):")
print("   - Размер:", f"{len(corr_matrix_2)}x{len(corr_matrix_2[0])} (те же {len(all_numeric_fields)} признаков)")
print("   - Корреляции стали более стабильными и надежными")
print("   - Удалено влияние выбросов и тождественных экземпляров")
print("   - Лучше отражает реальные взаимосвязи между признаками")

print("\n3. ТРЕТЬЯ МАТРИЦА (после всех преобразований):")
print("   - Размер: 3x3 (уменьшенная размерность)")
print("   - Данные преобразованы в целочисленную форму")
print("   - Сохранена основная информация методом обобщенной сериализации")
print("   - Низкая корреляция между компонентами упрощает анализ")


ИТОГОВЫЙ АНАЛИЗ ИЗМЕНЕНИЙ КОРРЕЛЯЦИОННЫХ МАТРИЦ

1. ПЕРВАЯ МАТРИЦА (после нормировки данных):
   - Размер: 18x18 (18 признаков)
   - Показала исходные взаимосвязи между всеми признаками
   - Могла содержать шум от выбросов и дубликатов
   - Значения корреляции отражали исходное распределение данных

2. ВТОРАЯ МАТРИЦА (после очистки данных):
   - Размер: 18x18 (те же 18 признаков)
   - Корреляции стали более стабильными и надежными
   - Удалено влияние выбросов и тождественных экземпляров
   - Лучше отражает реальные взаимосвязи между признаками

3. ТРЕТЬЯ МАТРИЦА (после всех преобразований):
   - Размер: 3x3 (уменьшенная размерность)
   - Данные преобразованы в целочисленную форму
   - Сохранена основная информация методом обобщенной сериализации
   - Низкая корреляция между компонентами упрощает анализ


In [194]:
# Ячейка 15: Финальная статистика по преобразованиям
print("\n" + "=" * 70)
print("ФИНАЛЬНАЯ СТАТИСТИКА ПРЕОБРАЗОВАНИЙ")
print("=" * 70)

print(f"\nИСХОДНЫЕ ДАННЫЕ:")
print(f"  Количество записей: {len(coffee_records)}")
print(f"  Количество признаков: 11 (все поля CSV)")
print(f"  Типы данных: смешанные (числа, текст, дата, время)")

print(f"\nПОСЛЕ ПРЕОБРАЗОВАНИЯ В ЧИСЛОВЫЕ:")
print(f"  Количество записей: {len(all_numeric_data)}")
print(f"  Количество признаков: {len(all_numeric_fields)}")
print(f"  Все данные: числовые")

print(f"\nПОСЛЕ ОЧИСТКИ:")
print(f"  Количество записей: {len(final_cleaned_data)}")
print(f"  Удалено записей: {len(all_numeric_data) - len(final_cleaned_data)}")
print(f"  Сохранено: {(len(final_cleaned_data)/len(all_numeric_data))*100:.1f}% данных")

print(f"\nПОСЛЕ ПРЕОБРАЗОВАНИЙ:")
print(f"  Финальное количество записей: {len(reduced_data)}")
print(f"  Финальная размерность: {len(reduced_data[0])} признаков")
print(f"  Мощности финальных признаков: {reduced_powers}")

print(f"\nОБЩАЯ ЭФФЕКТИВНОСТЬ:")
initial_size = len(coffee_records) * 11  # исходные 11 полей
final_size = len(reduced_data) * len(reduced_data[0])  # финальные данные
print(f"  Исходный объем данных: {initial_size} значений")
print(f"  Финальный объем данных: {final_size} значений")
print(f"  Сжатие: {(1 - final_size/initial_size)*100:.1f}%")

print(f"\nФИНАЛЬНЫЕ КОМПОНЕНТЫ:")
for i, (name, power) in enumerate(zip(reduced_names, reduced_powers)):
    print(f"  Компонента {i+1}: {name} (мощность: {power})")


ФИНАЛЬНАЯ СТАТИСТИКА ПРЕОБРАЗОВАНИЙ

ИСХОДНЫЕ ДАННЫЕ:
  Количество записей: 3547
  Количество признаков: 11 (все поля CSV)
  Типы данных: смешанные (числа, текст, дата, время)

ПОСЛЕ ПРЕОБРАЗОВАНИЯ В ЧИСЛОВЫЕ:
  Количество записей: 3547
  Количество признаков: 18
  Все данные: числовые

ПОСЛЕ ОЧИСТКИ:
  Количество записей: 2732
  Удалено записей: 815
  Сохранено: 77.0% данных

ПОСЛЕ ПРЕОБРАЗОВАНИЙ:
  Финальное количество записей: 2732
  Финальная размерность: 3 признаков
  Мощности финальных признаков: [221, 1519, 7744]

ОБЩАЯ ЭФФЕКТИВНОСТЬ:
  Исходный объем данных: 39017 значений
  Финальный объем данных: 8196 значений
  Сжатие: 79.0%

ФИНАЛЬНЫЕ КОМПОНЕНТЫ:
  Компонента 1: Компонента_5 (мощность: 221)
  Компонента 2: Компонента_4 (мощность: 1519)
  Компонента 3: Компонента_3 (мощность: 7744)
